## Flask

- Super useful framework
- With a few lines of code, we easily create an endpoint
- Documentation here: http://flask.pocoo.org/docs/0.10/

In [ ]:
# pip install Flask (your machines should already have it installed)

from flask import Flask # here we import the Flask class
app = Flask(__name__) # we create an instance of the Flask class -> '__name__'

@app.route("/") # we use whats called a 'decorator' to tell Flask what URL should trigger our function
def hello():
    return "Hello World!" 


# only runs if the script is executed directly from the Python interpreter (and not used as an imported module)
if __name__ == "__main__":  

    app.debug = True # we want to get error messages
    app.run(host='0.0.0.0', port=8886) # make your endpoint publicly available 

- we move this code to its own file
- then we run it on our remote machine (execute it directly from the Python interpreter)
- remember to open up the port 
- and now we can see its results when we navigate to:
    - http://[YOUR_PUBLIC_DNS]:8886/

- HELLO WORLD!

In [ ]:
# let's add a function -> given a tag, return the most recent instagram image for the tag

@app.route('/igtag/<tag>')
def igtag(tag):
    
    # we can call a function that has all the logic (hint: #hedgehog is an awesome IG tag!)
    return get_ig_media(tag)
    

In [48]:
from random import randint

def get_ig_media(tag):
    
    client_id = ''
    client_secret = ''

    from instagram.client import InstagramAPI

    api = InstagramAPI(client_id=client_id, client_secret=client_secret)
    
    media, _next = api.tag_recent_media(33, 0, tag)
    
    # let's make it a bit more interesting
    choice = randint(0,33)
    return media[choice].get_standard_resolution_url()
    

In [37]:
# now let's make it VISUAL - return HTML instead of links

@app.route('/igtagviz/<tag>')
def igtagviz(tag):
    
    url = get_ig_media(tag)
    
    # instead of returning the link, let's return html back
    return '<img src="%s"/>' % url
    

## Slack

-> add a service integration -> https://itpmssd.slack.com/services/new?channel_id=C0889K54J
- Slack Integrations are a way to add automated services into our slack account
- We're going to make an "incoming webhook"

### Incoming Webhook

In [54]:
import requests
import json

# here's our webhook url -> effectively gives us the "address" to send to the chosen slack channel
# need to get this from the slack website - 
webhook_url = ""

# something simple first
payload = {"text":'hi hi hi', "username":'whoa'}
r = requests.post(webhook_url, data={"payload":json.dumps(payload)})


In [52]:
# now lets send our incoming webhook an image!

cur_url = get_ig_media('hedgehog')
payload = {"text":cur_url}
r = requests.post(webhook_url, data={"payload":json.dumps(payload)})

### Slash Command

In [ ]:
# a slack /slash command gives us the ability to operate via command line on slack

- we need to go through all the setup steps (slack website/form)
- where it says 'integration settings':
    - input your URL (where your endpoint is running)
    - choose method (I'm using GET not POST)
    - Grab your token
    - enter autocomplete text

In [ ]:
# this is the function that will run on our server, that will receive the request from slack -> and return a response

@app.route('/ig',methods=["GET"])
def ig_slack_command():
     
    webhook_url = ""

    tag = requests.args.get('text',None)
    print tag
    
    if not tag:
        payload = {"text":'send me a tag!!', "username":'ig-bot'}
        r = requests.post(webhook_url, data={"payload":json.dumps(payload)})
    else:
        cur_url = get_ig_media('hedgehog')
        print cur_url
        payload = {"text":cur_url, "username":'ig-bot'}
        r = requests.post(webhook_url, data={"payload":json.dumps(payload)})
    
        
